In [4]:
# Libraries
# DATA MANAGEMENT
import pandas as pd
import numpy as np
from collections import Counter

# VISUALIAZATION
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# FILE SYSTEM
from os import listdir

In [5]:
def read_CSV_in_years(YEARS = ('2019', '2020', '2021', '2022', '2023')):
    '''
    Parameters
    -----------
    YEARS        : Years to include/open.

    Parameters
    -----------
    Opens all the MRT train csv files within the provided years.
    Call cleanCSV() to open, clean, and return a dataframe.

    Return   
    ------------
    pd.Dataframe : All the cleaned CSVs compiled into a single dataframe.
    '''
    compiled_df = []
    FILES = listdir('MTR Parsed')

    # OPEN AND STORE ALL THE TARGET CSV FILES
    for fileName in FILES:
        if fileName[-4:] == '.csv' and fileName[:4] in YEARS:
            compiled_df.append(clean_CSV(fileName))
    
    return pd.concat(compiled_df)

In [6]:
def clean_CSV(fileName, min_traffic_threshold = 0):
    '''
    Parameters
    -----------
    fileName               : String name of the .csv file.
    min_traffic_threshold  : Stations records with daily traffic below the threshold will be dropped.

    Parameters
    -----------
    Open the csv file and drop all records of stations with little/no activity for the entire day.

    Return   
    ------------
    pd.Dataframe : The cleaned csv data.
    '''
    df = pd.read_csv('MTR Parsed/' + fileName)
    # REMOVE INDEX COL
    df = df.drop(df.columns[0], axis=1)

    # CHECK THE TRAFFIC ACTIVITY OF PER STATION EVERYDAY 
    '''
    activity_check = df[['Day', 'Station_No', 'Net_Traffic']]
    activity_check = activity_check.groupby(['Day', 'Station_No']).sum()
    
    # KEEP ONLY THE STATION RECORDS THAT HAD ACTIVITY FOR THAT DAY
    records_to_keep = activity_check[activity_check.values > min_traffic_threshold]
    df = df.set_index(['Day', 'Station_No']).join(records_to_keep, lsuffix='' , rsuffix='_ActivityCheck')
    df = df.dropna(axis=0)
    df = df.rename(columns={df.columns[-1]:'Station_Total_Day_Traffic'})
    '''
    
    df['Date'] = pd.to_datetime(df['Date'])
    # df = df.reset_index()
    
    return df

### Contents

The data reflects a sorting order of...
<ol>
    <li>Date</li>
    <li>Time</li>
    <li>Station_No</li>
</ol>

In [7]:
df = read_CSV_in_years()

# RESET THE INDEX SINCE THERE WERE MULTPLE CSV FILES COMBINED, THUS DUPLICATION OF INDICES
df = df.reset_index().drop(columns='index')
df.tail()

,Date,Time,Station_Name,Station_No,Entry,Exit,Net_Traffic,Year,Month,Day,Hour,Weekday,Is_Holiday
531084,2023-12-31,02:00 - 02:59,Guadalupe,8,0,0.0,0.0,2023,12,31,2,6,True
531085,2023-12-31,02:00 - 02:59,Buendia,9,0,0.0,0.0,2023,12,31,2,6,True
531086,2023-12-31,02:00 - 02:59,Ayala Ave,10,0,0.0,0.0,2023,12,31,2,6,True
531087,2023-12-31,02:00 - 02:59,Magallanes,11,0,0.0,0.0,2023,12,31,2,6,True
531088,2023-12-31,02:00 - 02:59,Taft,12,0,0.0,0.0,2023,12,31,2,6,True


In [8]:
print(f'Entries: {df.shape}')

Entries: (531089, 13)


In [9]:
df.isnull().any()

Date            False
Time            False
Station_Name    False
Station_No      False
Entry           False
Exit            False
Net_Traffic     False
Year            False
Month           False
Day             False
Hour            False
Weekday         False
Is_Holiday      False
dtype: bool

# Feature modification

In [10]:
def convertToString(num : int):
    if(num < 10):
        return '0' + str(num)
    else:
        return str(num)

def joinNumsToString(df : pd.DataFrame, cols : list[str], separator = '_'):
    colName = cols[0] + separator + cols[1]
    df[colName] = df[cols[0]].apply(lambda x: convertToString(x)) +  separator + df[cols[1]].apply(lambda x: convertToString(x))

#### Timeseries string pairing
Optional step to better graph timeseries data. 

In [11]:
joinNumsToString(df, ['Year', 'Month'])
joinNumsToString(df, ['Month', 'Hour'])
joinNumsToString(df, ['Weekday', 'Hour'])
joinNumsToString(df, ['Day', 'Hour'])

In [12]:
# Change Mon -> Sun range to 1 -> 7
df['Weekday'] += 1

# Incomplete records
### Dropping `Exit` and `Net_Traffic`
The data provided by MTR is acknowledged to be incomplete. Notes were left inside the xlsx files often marking that `Entry` was correct as opposed to `Exit`. 

The graphs below show that there was nearly always greater `Entry` than `Exit`. <br>
Subtracting the 2, does not result in a zero difference, rather it is seen that the difference inidcates there is significantly more entries recorded.<br>
Extreme outliers are also present hence utilizing a single metric instead of both will be the ideal approach.

### To maintain consistency, `Entry` will be the label/value used henceforth

In [13]:
df.drop(columns=['Exit', 'Net_Traffic'], inplace=True)

# Optimize formats
Safe to lower the int64 and float64 formats. They are within int16 range and entry rate are whole numbers.

This reduces memory usage from original 67.3 MB to ~35MB

In [14]:
convertCols = df.select_dtypes(['int', 'float']).columns
df[convertCols].max()

Station_No      12
Entry         7445
Year          2023
Month           12
Day             31
Hour            23
Weekday          7
dtype: int64

In [15]:
for col in convertCols:
    if df[col].max() <= 127:
        df[col] = df[col].astype('int8')
    elif df[col].max() <= 32767:
        df[col] = df[col].astype('int16')
    else:
        print(f"{col} data out of range")
        assert False
        
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531089 entries, 0 to 531088
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Date          531089 non-null  datetime64[ns]
 1   Time          531089 non-null  object        
 2   Station_Name  531089 non-null  object        
 3   Station_No    531089 non-null  int8          
 4   Entry         531089 non-null  int16         
 5   Year          531089 non-null  int16         
 6   Month         531089 non-null  int8          
 7   Day           531089 non-null  int8          
 8   Hour          531089 non-null  int8          
 9   Weekday       531089 non-null  int8          
 10  Is_Holiday    531089 non-null  bool          
 11  Year_Month    531089 non-null  object        
 12  Month_Hour    531089 non-null  object        
 13  Weekday_Hour  531089 non-null  object        
 14  Day_Hour      531089 non-null  object        
dtypes: bool(1), datet

# Data Cleaning and Preprocessing

## Splicing to only Post-COVID Years [2022, 2023]
### Use `df_splice` from here onwards
To keep data relevant, we will be utilizing the post COVID records as there are still over 220,000 hourly records.

In [16]:
composition = df.groupby('Year').count()
composition['Percentage'] = composition.iloc[:,1] / composition.iloc[:,1].sum()
composition.rename(columns={"Time":"Number of Data Points"}, inplace=True)
composition[['Number of Data Points', 'Percentage']]

,Number of Data Points,Percentage
Year,,
2019,105144,0.197978
2020,84305,0.158740
2021,113880,0.214427
2022,113880,0.214427
2023,113880,0.214427


In [17]:
df_splice = df[df['Year'] >= 2022].copy()
df_splice = df_splice.reset_index().drop(columns='index')

df_splice.shape

(227760, 15)

## Excluding MRT Maintenance days

Scheduled maintanance periods will be excluded from `empty_records`. It will not be imputed to preserve that 0 values for the period.

<li><a href=https://primer.com.ph/blog/2022/03/18/dotr-mrt-3-to-pause-operation-during-holy-week-2022/>April 13-17, 2022</a></li>
<li><a href=https://www.philstar.com/headlines/2023/04/04/2256862/list-mrt-lrt-and-pnr-schedules-during-holy-week-2023#:~:text=%22To%20give%20way%20to%20the,April%206%20until%20April%209%2C>April 6-9, 2023</a></li>

In [26]:
print(df_splice.info())
# print(df_splice.Hour.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227760 entries, 0 to 227759
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Date          227760 non-null  datetime64[ns]
 1   Time          227760 non-null  object        
 2   Station_Name  227760 non-null  object        
 3   Station_No    227760 non-null  int8          
 4   Entry         227760 non-null  int16         
 5   Year          227760 non-null  int16         
 6   Month         227760 non-null  int8          
 7   Day           227760 non-null  int8          
 8   Hour          227760 non-null  int8          
 9   Weekday       227760 non-null  int8          
 10  Is_Holiday    227760 non-null  bool          
 11  Year_Month    227760 non-null  object        
 12  Month_Hour    227760 non-null  object        
 13  Weekday_Hour  227760 non-null  object        
 14  Day_Hour      227760 non-null  object        
dtypes: bool(1), datet

In [30]:
'''
1. Impute rows from 4am to 11pm (where entry is zero)
where Hour >= 0 & Hour  <= 3 
where Entry = 0 

2. Exclude April 13-17 2022 and April 6-9 2023
where Date < 2022-04-13 & Date > 2022-04-17
where Date < 2023-04-06 & Date > 2023-04-09
'''

df_to_impute = df_splice[~(df_splice.Hour.between(0,3)) & (df_splice.Entry == 0) & ~(df_splice.Date.between('2022-04-13', '2022-04-17')) & ~(df_splice.Date.between('2023-04-06', '2023-04-09'))]
df_to_impute.tail()

,Date,Time,Station_Name,Station_No,Entry,Year,Month,Day,Hour,Weekday,Is_Holiday,Year_Month,Month_Hour,Weekday_Hour,Day_Hour
227716,2023-12-31,23:00 - 23:59,Guadalupe,8,0,2023,12,31,23,7,True,2023_12,12_23,06_23,31_23
227717,2023-12-31,23:00 - 23:59,Buendia,9,0,2023,12,31,23,7,True,2023_12,12_23,06_23,31_23
227718,2023-12-31,23:00 - 23:59,Ayala Ave,10,0,2023,12,31,23,7,True,2023_12,12_23,06_23,31_23
227719,2023-12-31,23:00 - 23:59,Magallanes,11,0,2023,12,31,23,7,True,2023_12,12_23,06_23,31_23
227720,2023-12-31,23:00 - 23:59,Taft,12,0,2023,12,31,23,7,True,2023_12,12_23,06_23,31_23
